# DATA PREPARATION STARTING FROM DATAFRAMES

In [1]:
import pathlib
import numpy as np
import pandas as pd
# import torch

In [2]:
datestamp = "20241024-100738"
sim_folder = pathlib.Path("c:/Users/nikko/OneDrive/Documents/Uni/magistrale/tesi/simulations")
datafolder = sim_folder / datestamp / "data"

In [3]:
path_list = []
for i,path in enumerate(datafolder.iterdir()):
    if path.is_dir():
        df_path = path / f"{datestamp}_run{i+1}.txt"
        path_list.append(df_path)

In [4]:
df_list = []
for path in path_list:
    df_list.append(pd.read_csv(path))

In [5]:
num_exp = len(df_list)
Np = df_list[0].N.max()
num_steps = df_list[0].Time.max()

In [6]:
ppos = []
for df in df_list:
    Np = df.N.max()
    x = np.array(df.xpos).reshape(Np,-1)
    y = np.array(df.ypos).reshape(Np,-1)
    xy = np.stack([x,y],axis=2)
    ppos.append(xy)
# position = np.concatenate(ppos,axis=1)
position = np.stack(ppos,axis=3)
position = np.transpose(position,(3,1,0,2))


## Velocity calculation

In [7]:
timestep = 1e-3

vv = []
for df in df_list:
    df[["vx","vy"]] = df.groupby("N")[["xpos","ypos"]].diff()/timestep #velocity calculation
    vx = np.array(df.vx).reshape(Np,-1)
    vy = np.array(df.vy).reshape(Np,-1)
    vxy = np.stack([vx,vy],axis=2)
    vv.append(vxy)
# position = np.concatenate(ppos,axis=1)
vel = np.stack(vv,axis=3)
vel = np.transpose(vel,(3,1,0,2))

## Drag Coefficient

In [8]:
R = 2.
gamma = 6*np.pi*R*1e-3
gamma = np.repeat(gamma, Np)
gamma = gamma[:,np.newaxis]
gamma = np.tile(gamma,(num_exp,num_steps,1,1))

In [9]:
gamma.shape

(2, 1001, 10, 1)

In [10]:
data = {
    "position": position,
    "velocity": vel,
    "drag_coefficient": gamma
}